<a href="https://colab.research.google.com/github/Samir-atra/FCOS_Implementation/blob/main/model/FCOS_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# imports
# !pip3 install -U pip
# !pip3 install -U six numpy wheel packaging
# !pip3 install -U keras_preprocessing --no-deps
# !git clone https://github.com/tensorflow/tensorflow.git
# !cd tensorflow
# !pip install opencv-python
!pip install tf-models-official
# import tqdm
import tensorflow as tf
# import tensorflow_datasets as tfds
import tensorflow_models as tfm
import numpy as np
import json
import os
import cv2
import re
from operator import itemgetter
from Loss import IOULoss

In [ ]:
# download the dataset

# !wget http://images.cocodataset.org/zips/train2014.zip
# !wget http://images.cocodataset.org/zips/val2014.zip
# !wget http://images.cocodataset.org/annotations/annotations_trainval2014.zip
# !wget http://images.cocodataset.org/zips/test2014.zip
# !wget http://images.cocodataset.org/annotations/image_info_test2014.zip


--2024-10-11 15:01:42--  http://images.cocodataset.org/zips/train2014.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 16.182.70.129, 54.231.169.161, 54.231.140.185, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|16.182.70.129|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13510573713 (13G) [application/zip]
Saving to: ‘train2014.zip’

train2014.zip       100%[===================>]  12.58G  45.8MB/s    in 4m 23s  

2024-10-11 15:06:05 (49.0 MB/s) - ‘train2014.zip’ saved [13510573713/13510573713]



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# unzip the dataset files

# !pip install unzip
# !unzip "/content/drive/MyDrive/3rd ABAW Annotations.zip" -d /content/


In [3]:
# dataset paths

# for colab
train_imgs_path = "/content/drive/MyDrive/COCO/train2014/"
annotations = "/content/drive/MyDrive/COCO/annotations/"


#for library
# train_imgs_path = "C:/Users/1845718/Documents/FCOS_Implementation/train2014/train2014/"
# val_imgs_path = "C:/Users/1845718/Documents/FCOS_Implementation/val2014/val2014/"
# test_imgs_path = "C:/Users/1845718/Documents/FCOS_Implementation/test2014/test2014/"
# annotations = "C:/Users/1845718/Documents/FCOS_Implementation/annotations_trainval2014/annotations/"

# for laptop
# train_imgs_path = "/home/samer/Desktop/Beedoo/FCOS/FCOS_Implementation/COCO2014/train2014/"
# val_imgs_path = "/home/samer/Desktop/Beedoo/FCOS/FCOS_Implementation/COCO2014/val2014/"
# test_imgs_path = "/home/samer/Desktop/Beedoo/FCOS/FCOS_Implementation/COCO2014/test2014/"
# annotations = "/home/samer/Desktop/Beedoo/FCOS/FCOS_Implementation/COCO2014/annotations/"

In [4]:
images_dict = dict()
image_list = []
counter = 0
# load the images
for file in sorted(os.listdir(train_imgs_path)):
    if counter < 600:
        image_path = os.path.join(train_imgs_path, file)
        image = cv2.imread(image_path, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (800, 1024))
        image = np.array(image)
        image = image.astype('float32')
        image /= 255
        if len(image.shape) < 3:
            continue
        # plt.imshow(image)
        # plt.show()
        file_key = str(file).split('.')[0]
        imageid = re.findall("[0-9]+", file_key)
        imageid = str(imageid[1]).lstrip("_0")
        images_dict[f'{imageid}'] = image           # create a dictionary of the image titles and themselves
        image_list.append(image)
        counter += 1
    else:
        break

In [5]:
def concat(list1, list2):
    if type(list1[0]) == list and type(list2) == list:
        alist = list1
        alist.append(list2)
    elif type(list2) == int:
        alist = [list1]
        alist.append(list2)
    return alist

# check if need to put all the boxes of a single image in a list instead of keeping them separate.
# concat([1.08, 187.69, 611.59, 285.84],51)

In [6]:
labels_list = []
bounding_boxes = []
counter = 0
dont_care = [0,0,0,0,0,0,0,0,0]
annotation_path = os.path.join(annotations, "instances_train2014.json")
boxes = dict()
counter = 0

with open(annotation_path, "r") as f:
    json_obj = json.load(f)
    sorted_imageids = sorted(json_obj["annotations"], key=itemgetter("image_id"))
# draw the bounding boxes
    for line in sorted_imageids:      # iterate over the lines of the label file, where each line is a box with it's label
        if counter < 500:
            if not boxes.get(line["image_id"]):
                boxes[line["image_id"]] = [concat(line["bbox"], line["category_id"])]
            elif boxes.get(line["image_id"]):
                boxes[line["image_id"]] = concat(boxes[line["image_id"]], concat(line["bbox"], line["category_id"]))
            counter += 1

    print(boxes)



{9: [[[1.08, 187.69, 611.59, 285.84], 51], [[311.73, 4.31, 319.28, 228.68], 51], [[249.6, 229.27, 316.24, 245.08], 56], [[0.0, 13.51, 434.48, 375.12], 51], [[376.2, 40.36, 75.55, 46.53], 55], [[465.78, 38.97, 58.07, 46.67], 55], [[385.7, 73.66, 84.02, 70.51], 55], [[364.05, 2.49, 94.76, 71.07], 55]], 25: [[[385.53, 60.03, 214.97, 297.16], 25], [[53.01, 356.49, 132.03, 55.19], 25]], 30: [[[204.86, 31.02, 254.88, 324.12], 64], [[237.56, 155.81, 166.4, 195.25], 86]], 34: [[[0.96, 20.06, 441.23, 379.15], 24]], 36: [[[0.0, 50.12, 457.68, 430.35], 28], [[167.58, 162.89, 310.61, 465.19], 1]], 49: [[[162.57, 226.56, 130.41, 184.43], 19], [[81.7, 244.94, 83.71, 158.2], 19], [[203.39, 260.43, 65.69, 73.04], 1], [[118.43, 261.32, 56.91, 62.93], 1], [[119.34, 334.21, 11.94, 33.57], 1], [[201.77, 426.19, 70.84, 47.23], 64], [[284.34, 333.4, 10.88, 27.43], 1], [[191.0, 334.37, 7.16, 23.41], 1], [[346.76, 333.4, 14.8, 9.22], 1]], 61: [[[261.2, 205.92, 48.08, 57.24], 1], [[393.39, 210.34, 32.53, 39.74

In [8]:
# model backbone
# FCOS uses ResNeXt not his one.

resnet = tfm.vision.backbones.ResNet(
            model_id = 50,
            depth_multiplier = 1.0,
            replace_stem_max_pool = False,
            scale_stem = True,
            activation = "relu",
            bn_trainable = True)

exp_config = tfm.core.exp_factory.get_exp_config('resnet_imagenet')
exp_config.task.model.num_classes = 8
exp_config.task.model.input_size = list(1024,800,3)
exp_config.task.model.backbone.resnet.model_id = 50
batch_size = 16


TypeError: Inputs to a layer should be tensors. Got '<bound method Kernel.raw_input of <google.colab._kernel.Kernel object at 0x7f95bbf07040>>' (of type <class 'method'>) as input for layer 'res_net_2'.

In [ ]:
tfm.vision.decoders.FPN(
    input_specs: Mapping[str, tf.TensorShape],
    min_level = 3,
    max_level = 7,
    num_filters = 256,
    fusion_type = 'sum',
    use_separable_conv = False,
    use_keras_layer = False,
    activation = 'relu',
    use_sync_bn = False,
    norm_momentum = 0.99,
    norm_epsilon = 0.001,
    kernel_initializer = 'VarianceScaling',
    kernel_regularizer = None,
    bias_regularizer = None,
)

In [ ]:
# loss function

focal = tf.keras.losses.CategoricalFocalCrossentropy(
    alpha = 0.25,
    gamma = 2,
)


In [10]:
model.compile(optimizer = tf.keras.optimizers.SGD(learning_rate = 0.01,
              weight_decay = 0.0001,
              momentum = 0.9),
              loss = tf.keras.loss(IOULoss+focal)

              )

NameError: name 'model' is not defined

In [ ]:

def schedule(epoch, lr):
  if epoch == 60000 or epoch == 80000:
    lr = lr / 10
    return lr

sched = tf.keras.callbacks.LearningRateSchedualer(
    schedule
)

In [ ]:
model.fit(epochs = 90000,
          batch_size = 16,
          callbacks = [sched],

          )